# yt-napari 

```
pip install napari[pyQT5] yt-napari dask[distributed]
```

or if you care about QT versions

```
pip install napari[pyQT6] yt-napari 
```

one other plugin:

```
pip install napari-simpleitk-image-processing
```


In [1]:
import napari 
from yt_napari.viewer import Scene
import yt 

viewer = napari.Viewer()

yt : [INFO     ] 2025-07-17 10:40:33,295 Parameters: current_time              = 0.0060000200028298
yt : [INFO     ] 2025-07-17 10:40:33,296 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2025-07-17 10:40:33,297 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-07-17 10:40:33,298 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2025-07-17 10:40:33,299 Parameters: cosmological_simulation   = 0
Parsing Hierarchy : 100%|███████████████████| 173/173 [00:00<00:00, 9386.26it/s]
yt : [INFO     ] 2025-07-17 10:40:33,337 Gathering a field list (this may take a moment.)


In [ ]:

ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030")

yt_scene = Scene()

left_edge = ds.domain_center - ds.arr([10, 10, 10], 'kpc')
right_edge = ds.domain_center + ds.arr([10, 10, 10], 'kpc')
yt_scene.add_region(viewer,
                    ds,
                    ("enzo", "Density"),
                    left_edge = left_edge,
                    right_edge = right_edge,
                    resolution=(600, 600, 600),
                    colormap="magma",
                    contrast_limits=(3.4, 6.4))

yt_scene.add_region(viewer,
                    ds,
                    ("enzo", "Temperature"),
                    left_edge = left_edge,
                    right_edge = right_edge,
                    resolution=(300, 300, 300),
                    colormap="yellow",
                    contrast_limits=(4., 6.),
                    opacity=0.6)

Can also: 

* load from JSON file
* load from GUI
  
https://yt-napari.readthedocs.io/en/latest/quickstart.html#loading-a-json-file-from-the-napari-gui

## Loading timeseries slices with dask

In [2]:
from dask.distributed import Client

In [3]:
c = Client(n_workers=2, threads_per_worker=1)

In [4]:
from yt_napari import timeseries

viewer.layers.clear() 

slc = timeseries.Slice(("enzo", "Density"), "x", resolution=(400, 400))
file_pattern = "enzo_tiny_cosmology/DD????/DD????"
timeseries.add_to_viewer(viewer,
                         slc, 
                         file_pattern=file_pattern, 
                         load_as_stack=True,
                         use_dask=True,
                         contrast_limits=(-1, 2),
                         colormap = 'magma',
                         name="Lazy density")

Parsing Hierarchy : 100%|██████████| 201/201 [00:00<00:00, 9415.72it/s]


## The napai plugin ecosystem

https://yt-napari.readthedocs.io/en/latest/tutorials.html
* Interactive visualization with the wider plugin ecosystem
* Image segmentation and classification with additional plugins

https://www.napari-hub.org/

In [8]:
viewer.layers.clear()

In [9]:
ds = yt.load_sample("Enzo_64")

yt_scene = Scene()

left_edge = ds.domain_center - ds.arr([10, 10, 10], 'kpc')
right_edge = ds.domain_center + ds.arr([10, 10, 10], 'kpc')
yt_scene.add_region(viewer,
                    ds,
                    ("gas", "density"),
                    left_edge = ds.domain_left_edge,
                    right_edge = ds.domain_right_edge,
                    rescale=True,
                    resolution=(256, 256, 256),                    
                    colormap="magma",)

yt_scene.add_region(viewer,
                    ds,
                    ("gas", "temperature"),
                    left_edge = ds.domain_left_edge,
                    right_edge = ds.domain_right_edge,
                    resolution=(512, 512, 512),                    
                    colormap="viridis",)

yt : [INFO     ] 2025-07-17 10:46:04,354 Sample dataset found in '/home/chavlin/data/yt_data/yt_sample_sets/Enzo_64/DD0043/data0043'
yt : [INFO     ] 2025-07-17 10:46:04,621 Parameters: current_time              = 645.81707236914
yt : [INFO     ] 2025-07-17 10:46:04,622 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2025-07-17 10:46:04,624 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-07-17 10:46:04,628 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2025-07-17 10:46:04,630 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2025-07-17 10:46:04,631 Parameters: current_redshift          = 0.0013930880640796
yt : [INFO     ] 2025-07-17 10:46:04,633 Parameters: omega_lambda              = 0.7
yt : [INFO     ] 2025-07-17 10:46:04,634 Parameters: omega_matter              = 0.3
yt : [INFO     ] 2025-07-17 10:46:04,636 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2025-07-17 10:46:04,639 Parameters: hub

Napari status bar display of label properties disabled because https://github.com/napari/napari/issues/5417 and https://github.com/napari/napari/issues/4342


simple-itk:

* Otsu multiple thresholds (drop to 128 bins, bump levels to 5)
* Label statistics